## **References** -
- **Notebook:** https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing
- **Hugging Face:** https://huggingface.co/unsloth/Meta-Llama-3.1-8B-bnb-4bit

## **Installations**

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

## **Loading the Model**

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

### **Loading the Dataset**

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the CSV file using pandas
train_excel_file = pd.read_csv('train_data.csv')
val_excel_file = pd.read_csv('val_data.csv')

# Convert the pandas DataFrame to a datasets Dataset
train_dataset = Dataset.from_pandas(train_excel_file)
val_dataset = Dataset.from_pandas(val_excel_file)

# Now you have two separate datasets for training and validation
print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

### ***Data Prep***

In [ ]:
legal_prompt = """You are tasked with generating detailed summaries from the perspectives of both the prosecution and the defense for the following legal judgment text. Each summary should be comprehensive and highlight key aspects relevant to each side.\n\n

    ### judgement:\n{}\n\n
    ### summary:\n{}\n\n
    """
legal_prompt_prosecution = """You are required to generate a detailed summary of the legal judgment text from the perspective of the prosecution. Focus on key arguments, evidence, and legal points that strengthen the prosecution's case.

### Judgment:\n{}\n\n
### Prosecution Summary:\n{}\n\n
"""
legal_prompt_defence = """You are required to generate a detailed summary of the legal judgment text from the perspective of the defence. Focus on key arguments, evidence, and legal points that strengthen the defence's case.

### Judgment:\n{}\n\n
### Defence Summary:\n{}\n\n
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    judgement = examples["Judgement"]
    summary       = examples["Perspective-based Summary"]
    # summary = examples["prosecutor_pov"]
    # summary = examples["defense_pov"]

    texts = []
    for judgement, summary in zip(judgement, summary):
        text = legal_prompt.format(judgement, summary) + EOS_TOKEN
        # text = legal_prompt_prosecution.format(judgement, summary) + EOS_TOKEN
        # text = legal_prompt_defence.format(judgement, summary) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# Mapping
train_dataset = train_dataset.map(formatting_prompts_func, batched = True)
val_dataset = val_dataset.map(formatting_prompts_func, batched = True)

### **Train the Model**

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

repository = "user_name/model_name"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = repository,
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

### **Saving and load the model**

In [ ]:
# Push model and tokenizer to Hugging Face Hub
trainer.create_model_card()
trainer.push_to_hub(repository)